# ADK Callbacks (Doc-faithful, concise)

Callbacks let you observe, modify, or override an agent’s behavior at precise points.

## What you’ll learn
- Six callbacks and when they run
- How to let default behavior continue vs. override it
- Minimal wiring for each type


## Where callbacks fire

```mermaid
flowchart LR
  U[User] --> R[Runner]
  R --> A[Agent]
  A -->|before_agent| A
  A -->|before_model| L[LLM]
  L -->|after_model| A
  A -->|before_tool| T[Tool]
  T -->|after_tool| A
  A -->|after_agent| R --> U
```

- **before_agent / after_agent** — wrap the agent’s main work for a turn.
- **before_model / after_model** — around the LLM request/response.
- **before_tool / after_tool** — around each tool call.


## Types and override rules

| Callback | Typical purpose | Let default proceed | Override / Replace |
|---|---|---|---|
| `before_agent` | pre-checks, setup state | return `None` | return `types.Content` (skip agent run) |
| `after_agent` | post-format, final edits | return `None` | return `types.Content` (replace final) |
| `before_model` | guardrails, prompt tweaks | return `None` | return `LlmResponse` (skip LLM) |
| `after_model` | sanitize/reshape LLM output | return `None` | return `LlmResponse` (replace LLM) |
| `before_tool` | auth/args/rate limit | return `None` | return `dict` (skip tool, use this) |
| `after_tool` | normalize tool result | return `None` | return `dict` (replace tool result) |

Tip: Update turn/session state via the provided context objects; don’t mutate session state directly during a run.


## Minimal wiring (all six callbacks)
Below: concise handlers and an `LlmAgent` that uses them. Add your tools later to see tool callbacks in action.


In [ ]:
from typing import Optional
from google.adk.agents import LlmAgent
from google.adk.agents.callback_context import CallbackContext
from google.adk.models import LlmRequest, LlmResponse
from google.genai import types

# -------- Agent lifecycle --------
def before_agent_cb(callback_context: CallbackContext) -> Optional[types.Content]:
    # Example: mark a trace id for this turn
    callback_context.state["temp:trace_id"] = "trace-001"
    return None  # allow agent to run

def after_agent_cb(callback_context: CallbackContext) -> Optional[types.Content]:
    # Example: keep default; you could return Content(...) to replace final
    return None

# -------- LLM interaction --------
def before_model_cb(callback_context: CallbackContext, llm_request: LlmRequest) -> Optional[LlmResponse]:
    # Example: prepend a concise system instruction
    sys_msg = types.Content(role="system", parts=[types.Part(text="Be concise.")])
    cfg = llm_request.config or types.GenerateContentConfig()
    cfg.system_instruction = sys_msg
    llm_request.config = cfg
    # Example skip path: if you wanted to block, return an LlmResponse(...) here
    return None

def after_model_cb(callback_context: CallbackContext, llm_response: LlmResponse) -> Optional[LlmResponse]:
    # Example: keep default; you could return a modified LlmResponse
    return None

# -------- Tool execution --------
def before_tool_cb(tool, args, tool_context):
    # Example: clamp an optional 'limit' argument if present
    if isinstance(args, dict) and "limit" in args:
        args["limit"] = min(int(args["limit"]), 5)
    return None  # allow tool call

def after_tool_cb(tool, args, tool_context, tool_response):
    # Example: attach a 'source' key if tool returns a dict
    if isinstance(tool_response, dict):
        tool_response.setdefault("source", getattr(tool, "name", "tool"))
        return tool_response  # replaces tool result
    return None

callbacks_agent = LlmAgent(
    model="gemini-2.5-flash",
    name="callbacks_agent",
    instruction="Answer helpfully in one short paragraph.",
    before_agent_callback=before_agent_cb,
    after_agent_callback=after_agent_cb,
    before_model_callback=before_model_cb,
    after_model_callback=after_model_cb,
    before_tool_callback=before_tool_cb,
    after_tool_callback=after_tool_cb,
    tools=[]
)
print("Agent ready:", callbacks_agent.name)

```mermaid
flowchart LR
   A[before_model] -->|return None| M[Call LLM]
   A -->|return LlmResponse| X[Skip LLM<br/>use returned response]
   M --> B[after_model]
   B -->|return None| C[Use LLM response]
   B -->|return LlmResponse| D[Replace response]
   

```mermaid
flowchart LR
   P[before_tool] -->|return None| T[Run tool]
   P -->|return dict| S[Skip tool<br/>use returned dict]
   T --> Q[after_tool]
   Q -->|return None| U[Use tool result]
   Q -->|return dict| V[Replace result]
   